# Process single cell morphology features for CellProfiler readouts - PyBaSiC and CellProfiler Cellpose plugin Method

## Import Libraries

In [1]:
import pathlib
import pandas as pd

from pycytominer import normalize, feature_select
from pycytominer.cyto_utils import cells, output

## Set up paths to CellProfiler directory and outputs

In [2]:
# Set file and directory constants
cp_dir = "../../../CellProfiler_pipelines"
output_dir = "../../data/Plate1/CellProfiler"

## Set up paths to sqlite files and outputs

In [3]:
# Set name and path of .sqlite file and path to metadata
sql_file_pbcellpose = "NF1_data_pybasic_cellpose_plate1.sqlite"
single_cell_file_pbcellpose = f"sqlite:///{cp_dir}/Analysis_Output/Plate1_Output/{sql_file_pbcellpose}"
platemap_file = f"{cp_dir}/Metadata/platemap_NF1_CP.csv"

# Set path with name for outputted data
sc_output_file_pbcellpose = pathlib.Path(f"{output_dir}/nf1_sc_pybasic_cellpose.csv.gz")
sc_norm_output_file_pbcellpose = pathlib.Path(f"{output_dir}/nf1_sc_norm_pybasic_cellpose.csv.gz")
sc_norm_fs_output_file_pbcellpose = pathlib.Path(f"{output_dir}/nf1_sc_norm_fs_pybasic_cellpose.csv.gz")

## Set up names for linking columns between tables in the database file

In [4]:
# Define custom linking columns between compartments
linking_cols = {
    "Per_Cytoplasm": {
        "Per_Cells": "Cytoplasm_Parent_Cells",
        "Per_Nuclei": "Cytoplasm_Parent_Nuclei",
    },
    "Per_Cells": {"Per_Cytoplasm": "Cells_Number_Object_Number"},
    "Per_Nuclei": {"Per_Cytoplasm": "Nuclei_Number_Object_Number"},
}

## Load in platemap

In [5]:
# Load platemap file
platemap_df = pd.read_csv(platemap_file)
platemap_df

,WellRow,WellCol,well_position,gene_name,genotype
0,C,6,C6,NF1,WT
1,C,7,C7,NF1,Null
2,D,6,D6,NF1,WT
3,D,7,D7,NF1,Null
4,E,6,E6,NF1,WT
5,E,7,E7,NF1,Null
6,F,6,F6,NF1,WT
7,F,7,F7,NF1,Null


## Set up `SingleCells` class from Pycytominer

In [6]:
# Instantiate SingleCells class
sc_pbcellpose = cells.SingleCells(
    sql_file=single_cell_file_pbcellpose,
    compartments=["Per_Cells", "Per_Cytoplasm", "Per_Nuclei"],
    compartment_linking_cols=linking_cols,
    image_table_name="Per_Image",
    strata=["Image_Metadata_Well", "Image_Metadata_Plate"],
    merge_cols=["ImageNumber"],
    image_cols="ImageNumber",
    load_image_data=True
)

/home/jenna/anaconda3/envs/4.process-nf1-features/lib/python3.8/site-packages/pycytominer/cyto_utils/util.py:61: UserWarning: Non-canonical compartment detected: per_cells, per_cytoplasm, per_nuclei
  warnings.warn(warn_str)


## Merge single cells 

In [7]:
# Merge single cells across compartments
anno_kwargs = {"join_on": ["Metadata_well_position", "Image_Metadata_Well"]}

sc_df_pbcellpose = sc_pbcellpose.merge_single_cells(
    platemap=platemap_file,
    **anno_kwargs,
)

# Save level 2 data as a csv
output(sc_df_pbcellpose, sc_output_file_pbcellpose)

print(sc_df_pbcellpose.shape)
sc_df_pbcellpose.head()

(257, 1207)


,Metadata_WellRow,Metadata_WellCol,Metadata_gene_name,Metadata_genotype,Metadata_ImageNumber,Metadata_Plate,Metadata_Well,Metadata_Cytoplasm_Parent_Cells,Metadata_Cytoplasm_Parent_Nuclei,Metadata_Cells_Number_Object_Number,...,Nuclei_Texture_Variance_DAPI_3_02_256,Nuclei_Texture_Variance_DAPI_3_03_256,Nuclei_Texture_Variance_GFP_3_00_256,Nuclei_Texture_Variance_GFP_3_01_256,Nuclei_Texture_Variance_GFP_3_02_256,Nuclei_Texture_Variance_GFP_3_03_256,Nuclei_Texture_Variance_RFP_3_00_256,Nuclei_Texture_Variance_RFP_3_01_256,Nuclei_Texture_Variance_RFP_3_02_256,Nuclei_Texture_Variance_RFP_3_03_256
0,C,6,NF1,WT,1,001,C6,1,7,1,...,1284.672174,1242.119627,341.377503,313.497763,311.175957,281.512948,95.963488,91.315173,102.977187,102.522472
1,C,6,NF1,WT,1,001,C6,2,6,2,...,1471.89699,1393.751556,630.872348,597.307265,602.902214,552.229859,104.83556,107.01408,117.485297,103.801332
2,C,6,NF1,WT,1,001,C6,3,9,3,...,910.200747,883.924202,220.350381,194.886524,193.937829,197.232001,94.887681,94.716182,92.987935,92.455347
3,C,6,NF1,WT,1,001,C6,4,10,4,...,1335.188235,1312.799663,375.982151,364.429794,367.293125,339.249043,624.960023,625.649014,611.611592,613.090462
4,C,6,NF1,WT,1,001,C6,5,11,5,...,1045.954375,969.975823,256.454939,259.625549,278.603785,246.547134,92.923097,93.491099,96.674695,92.564448


## Normalize data

In [8]:
# Normalize single cell data and write to file
normalize_sc_pbcellpose = normalize(
    sc_df_pbcellpose,
    method="standardize"
)

output(normalize_sc_pbcellpose, sc_norm_output_file_pbcellpose)

print(normalize_sc_pbcellpose.shape)
normalize_sc_pbcellpose.head()

(257, 1207)


,Metadata_WellRow,Metadata_WellCol,Metadata_gene_name,Metadata_genotype,Metadata_ImageNumber,Metadata_Plate,Metadata_Well,Metadata_Cytoplasm_Parent_Cells,Metadata_Cytoplasm_Parent_Nuclei,Metadata_Cells_Number_Object_Number,...,Nuclei_Texture_Variance_DAPI_3_02_256,Nuclei_Texture_Variance_DAPI_3_03_256,Nuclei_Texture_Variance_GFP_3_00_256,Nuclei_Texture_Variance_GFP_3_01_256,Nuclei_Texture_Variance_GFP_3_02_256,Nuclei_Texture_Variance_GFP_3_03_256,Nuclei_Texture_Variance_RFP_3_00_256,Nuclei_Texture_Variance_RFP_3_01_256,Nuclei_Texture_Variance_RFP_3_02_256,Nuclei_Texture_Variance_RFP_3_03_256
0,C,6,NF1,WT,1,001,C6,1,7,1,...,-1.027216,-1.010803,-0.085097,-0.109356,-0.153852,-0.187294,0.074890,0.051210,0.118889,0.122227
1,C,6,NF1,WT,1,001,C6,2,6,2,...,-0.833078,-0.846328,0.541380,0.518134,0.475344,0.402793,0.130204,0.149229,0.209502,0.130284
2,C,6,NF1,WT,1,001,C6,3,9,3,...,-1.415514,-1.399336,-0.347004,-0.371600,-0.406711,-0.371002,0.068183,0.072445,0.056499,0.058806
3,C,6,NF1,WT,1,001,C6,4,10,4,...,-0.974834,-0.934136,-0.010211,0.003253,-0.032818,-0.061445,3.372974,3.387409,3.295678,3.338734
4,C,6,NF1,WT,1,001,C6,5,11,5,...,-1.274748,-1.305996,-0.268872,-0.228465,-0.224103,-0.263509,0.055935,0.064796,0.079525,0.059493


## Feature selection

In [9]:
feature_select_ops = [
    "variance_threshold",
    "correlation_threshold",
    "blocklist",
]

feature_select_norm_sc_pbcellpose = feature_select(
    normalize_sc_pbcellpose,
    operation=feature_select_ops
)

output(feature_select_norm_sc_pbcellpose, sc_norm_fs_output_file_pbcellpose)

print(feature_select_norm_sc_pbcellpose.shape)
feature_select_norm_sc_pbcellpose.head()

(257, 447)


,Metadata_WellRow,Metadata_WellCol,Metadata_gene_name,Metadata_genotype,Metadata_ImageNumber,Metadata_Plate,Metadata_Well,Metadata_Cytoplasm_Parent_Cells,Metadata_Cytoplasm_Parent_Nuclei,Metadata_Cells_Number_Object_Number,...,Nuclei_Texture_InfoMeas1_RFP_3_03_256,Nuclei_Texture_InfoMeas2_DAPI_3_03_256,Nuclei_Texture_InfoMeas2_GFP_3_02_256,Nuclei_Texture_InfoMeas2_RFP_3_00_256,Nuclei_Texture_InfoMeas2_RFP_3_01_256,Nuclei_Texture_InfoMeas2_RFP_3_02_256,Nuclei_Texture_InfoMeas2_RFP_3_03_256,Nuclei_Texture_SumEntropy_DAPI_3_03_256,Nuclei_Texture_SumVariance_DAPI_3_03_256,Nuclei_Texture_SumVariance_RFP_3_03_256
0,C,6,NF1,WT,1,001,C6,1,7,1,...,-0.442760,0.083127,0.585675,-0.735158,-0.685079,0.104043,0.836933,0.820346,-0.871101,0.147591
1,C,6,NF1,WT,1,001,C6,2,6,2,...,0.289889,0.500007,1.218839,-0.207573,0.384508,1.033588,0.306124,1.023874,-0.702089,0.085247
2,C,6,NF1,WT,1,001,C6,3,9,3,...,0.396768,-0.115062,-0.068476,-0.277234,0.468193,1.257439,0.184648,0.592465,-1.327725,0.022870
3,C,6,NF1,WT,1,001,C6,4,10,4,...,-1.475681,0.474145,0.516295,1.028925,1.205727,1.300419,1.460771,0.850093,-0.770668,3.251453
4,C,6,NF1,WT,1,001,C6,5,11,5,...,0.184984,-0.030681,0.626728,0.332476,0.636206,0.593752,0.384545,0.718838,-1.236696,0.030306


---

### Visualize basic count statistics

In [10]:
sc_df_pbcellpose.Metadata_genotype.value_counts()

Null    182
WT       75
Name: Metadata_genotype, dtype: int64

In [11]:
pd.crosstab(sc_df_pbcellpose.Metadata_genotype, sc_df_pbcellpose.Metadata_Well)

Metadata_Well,C6,C7,D6,D7,E6,E7,F6,F7
Metadata_genotype,,,,,,,,
Null,0,23,0,32,0,59,0,68
WT,26,0,7,0,17,0,25,0
